Creates database for all of the quotes.

In [ ]:
#Basic basic beginning search algorithim to tell you which simpsons episode the quote is from
#database will be as follows
#szn #, episode #, link to transcript, transcript



from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import string
import re

#this function gets the season number from a transcript link, by parsing the link's string
#each link contains the season/episode number in it, so these functions extract them
def seasonNumber(link):
    start1 = link.find('season-')
    end = link.find('/', start1)
    seasonNo = (link[start1+7:end])
    return seasonNo

#same as above, parses the transcript link's string and returns the episode number
def episodeNumber(link):
    start1 = link.find('episode')
    end = link.find('-', start1+8)
    episodeNo = (link[start1+8:end])
    return episodeNo


my_url = 'https://subslikescript.com/series/The_Simpsons-96697'

#makes connection to url, then reads it, and closes it

uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()

#access the html and get a link for the transcript

page_soup = soup(page_html, "html.parser")

#seasons is a list of all seasons, in the HTML code for the site, each season category
#has the div class season, findAll makes a bs4 list of all of the occurances
seasons = page_soup.findAll("div",{"class":"season"})

#creating/openning a csv file in writing mode
out_filename = "simpsonsTranscriptsDraft4.csv"
f = open(out_filename, 'w')
    
# https://www.youtube.com/watch?v=XQgXKtPSzUI is a source I used to begin learning to web scrape
# along with p4e ch 12


#goal is to make list of each transcript link in a table

#for each class season in list of seasons

#the first season occurance is not useful/relevant so it is deleted, headers are written
del seasons[0]
f.write('Season number,Episode number,Link,Transcript \n')

#iterating through each seasons element, each element contains a link to the transcript
for season in seasons:
    count = 0
    episodeList = season.findAll('a')  #list of all of the a tags in each season
    #iterating through each episode of each season, using the a tags
    #a tags are how the site groups the epsidoes
    while count < len(episodeList):
            #the 'href' part of the code contains the second half of the link to the transcript
            episodeExt = season.select('a')[count]['href']
            #completing the link here
            episodeLink = 'https://subslikescript.com' + episodeExt
            count = count + 1
            #passing link through my fxns to get the episode/season #
            seasonNo = seasonNumber(episodeLink)
            episodeNo = episodeNumber(episodeLink)
            #writing in the information found into csv file variable f
            f.write(seasonNo + ',' + episodeNo + ',' + episodeLink + ',')
            #set up the try/except for debugging, almost no errors now, going to try 
            #to eliminate all of them by the final checkpoint
            try:
                #as above, opens link, scans through html
                uClient1 = uReq(episodeLink)
                episode_html = uClient1.read()
                episode_soup = soup(episode_html, "html.parser")
                #picks up the script in the html
                script = episode_soup.find("div", {"class":"full-script"})
                #convert to string, replace some of the html lingo leftover, remove punctuation
                cleanScript = str(script)
                cleanScript = cleanScript.replace('<br/>', ' ')
                #punctuation removal taken from p4e chapter 6
                cleanScript = cleanScript.translate(cleanScript.maketrans('','',string.punctuation))
                #lowercase everything for ease of search, takeaway newline character
                cleanScript = cleanScript.lower()
                cleanScript = cleanScript.replace('\n', ' ')
                #remove that note logo, which can't be encoded into csv file
                cleanScript = cleanScript.replace('♪', ' ')  
                #clean up beggining and end of the string
                cleanScript = re.sub("^div classfullscript", "", cleanScript)
                cleanScript = re.sub('div$', '', cleanScript)
                #write in the finish script and then make newline for the next row of info
                f.write(cleanScript + '\n')
            except:
                #for debugging
                f.write('error \n')  
#close url reader, close file
uClient1.close()
f.close()

#https://www.youtube.com/watch?v=7DG3kCDx53c was also helpful to introuce regex
#https://www.youtube.com/watch?v=q5uM4VKywbA was useful in introducing csv module

Basic Searching Engine

In [4]:
import csv

#receives input for a search phrase
#opens up databse in csv reading format, uses csv.reader to read it
while True:
    searching = input('Type in a quote, all lowercase, no punctuation, that you want to locate: \nEnter "done" to stop. ')
    if searching == 'done':
        break
    with open('simpsonsTranscriptsDraft4.csv', 'r') as csv_file:
            csv_reader = csv.reader(csv_file)
            #creates variable named found, this switches on once the quote has been found
            found = 'n'
            #iterates through each row
            for line in csv_reader:
                listOccurances = []
                #insures that there is no error in the row; that each header is completed
                if len(line) == 4:
                    #if search phrase is located, print out which episode number/season it was found in
                    if searching in line[3]:
                        print('quote found in szn ' + line[0] + ' episode ' + line[1])
                        #changes found to y
                        found = 'y'
            #if it hasn't been found, print saying so
            if found == 'n':
                print('Quote not found')

Type in a quote, all lowercase, no punctuation, that you want to locate: 
Enter "done" to stop. hi
quote found in szn 1 episode 1
quote found in szn 1 episode 2
quote found in szn 1 episode 3
quote found in szn 1 episode 4
quote found in szn 1 episode 5
quote found in szn 1 episode 6
quote found in szn 1 episode 7
quote found in szn 1 episode 8
quote found in szn 1 episode 9
quote found in szn 1 episode 10
quote found in szn 1 episode 11
quote found in szn 1 episode 12
quote found in szn 1 episode 13
quote found in szn 2 episode 1
quote found in szn 2 episode 2
quote found in szn 2 episode 3
quote found in szn 2 episode 4
quote found in szn 2 episode 5
quote found in szn 2 episode 6
quote found in szn 2 episode 7
quote found in szn 2 episode 8
quote found in szn 2 episode 9
quote found in szn 2 episode 10
quote found in szn 2 episode 11
quote found in szn 2 episode 12
quote found in szn 2 episode 13
quote found in szn 2 episode 14
quote found in szn 2 episode 15
quote found in szn 2 ep

Type in a quote, all lowercase, no punctuation, that you want to locate: 
Enter "done" to stop. done


__Beta testing:__


Moving beyond basic search, process documented:   Just brainstorming ideas to match similar but not exact quotes


Taking your advice I looked into using fuzzywuzzy to find quotes that would have been close matches.

I turned each transcript into a list, list transcripts...  ex: [trans1, trans2, trans3.....]

Then did a simple matching function to try to come up with potential matches:

    from fuzzywuzzy import process
    def getMatches(query):
        results = process.extractOne(query, listTranscripts)
        return results
    
The problem I ran into was that this method only measures how similar two strings are together, and I was comparing a quote-string to transcript-strings. eg. comparing 'mary had a' to 'mary had a little lamb little lamb little lamb'

Thus in order to use this sort of function, I would need to essentially redo the databse program to include a list, for each transcript, of every quote.  

It would be like:

    QuoteListTrans1 = [quote1, quote2, quote3....]
    QuoteListTrans2 = [quote1, quote2, quote3....]
    For all transcripts

And then make that a list of those lists so I would then have a list of quote lists.

    SearchList = [QuoteListTrans1, QuoteListTrans2, QuoteListTrans3...]


I could then make the function like this

    def getMatches(query):
        listResult = []
        for quotelist in SearchList:
            result = process.extractOne(query, quotelist)
            listResult.append(result)
            #creates list of tuples (best quote-match from that episode, fuzzy-match rating)

        bestMatch = max(listResult, key = lambda item:item[1])
        #find quote with highest fuzzy-match rating
        bestQuote = bestMatch[0]
        return bestQuote
    
    
so I would compare the user's query to every quote in every transcript... then I would return the best matched quote

now that I have the exact quote - I just need to tell the user which episode/szn it came from...

I can use my original function to find this again.

    for line in csv_reader:
        print('A similar quote was found, the quote is: ' + bestQuote )
        if len(line) == 4:
           if bestQuote in line[3]:
                print('This similar-quote match found in szn ' + line[0] + ' episode ' + line[1])




As you can see, (and yes, you did warn me this in the beginning) when I start to look for potentially matching quotes it drastically increases the amount of potential code breaks, as well as room for error and misunderstanding of the user.

The major reason why is just the sheer number of quotes.  When making each transcript into a list of quotes, I am now dealing with potentially tens of thousands of quotes.  So there are going to be many many more ways for some quotes to not be extracted correctly, or to be cut off, misread, etc.



The only other idea I had is to change the users input into a regex type formula to search through transcripts that way.  This would solve the case of if the user put a typo in or maybe used a punctuation mark when told not to.  But it wouldn't solve the problem if they couldn't think of the exact quote to begin with.


__Conclusion, scope of project:__
    
    Created searchable database of all simpons transcripts, with minimal errors.
    
    Created basic exact user-quote matching and episode/season retrival.
    
    Identified ways to expand the program to search for close-matches, and to lead the user the quote found/episode number